<a href="https://colab.research.google.com/github/vinhpeters/tf-exam/blob/main/08_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
   !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-04-19 17:53:05--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-19 17:53:05 (62.8 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [43]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [44]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-04-19 17:53:05--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 74.125.134.128, 74.125.196.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-04-19 17:53:05 (122 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [45]:
unzip_data('/content/nlp_getting_started.zip')

## Data exploration

In [46]:
import pandas as pd

train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [47]:
train_df_shuffled = train_df.sample(frac=1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [48]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [49]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [50]:
len(train_df), len(test_df)

(7613, 3263)

In [51]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text,target = row
  print(f'Target: {target}','Real disaster' if target > 0 else 'Not a disaster')
  print(f'Text:\n{text}\n')

Target: 1 Real disaster
Text:
@helene_yancey GodsLove &amp; #thankU my sister Helene for RT of NEW VIDEO http://t.co/cybKsXHF7d The Coming Apocalyptic US Earthquake &amp; Tsunami

Target: 1 Real disaster
Text:
@aelinrhee a group of mascara smeared girls rioting will be horrific I think

Target: 0 Not a disaster
Text:
when you got an extension cord that extends from your bed to your bath tub ?? lets pray I don't electrocute myself

Target: 1 Real disaster
Text:
@HowardU If 90BLKs&amp;8WHTs colluded 2 take WHT F @USAgov AUTH Hostage&amp;2 make her look BLK w/Bioterrorism&amp;use her lgl/org IDis ID still hers?

Target: 0 Not a disaster
Text:
IK Only Troll His Pol Rivals Never Literally Abused Them Or Attacked Their Families. While All Of Them Literally Abuse IK. Loosers



In [52]:
from sklearn.model_selection import train_test_split


In [53]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'], train_df_shuffled['target'], test_size=0.1, random_state=42)

In [54]:
len(train_sentences)

6851

In [55]:
train_sentences[:10],train_labels[:10]

(5921    @mogacola @zamtriossu i screamed after hitting...
 3904              Imagine getting flattened by Kurt Zouma
 2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
 3718    @shakjn @C7 @Magnums im shaking in fear he's g...
 1667    Somehow find you and I collide http://t.co/Ee8...
 4435    @EvaHanderek @MarleyKnysh great times until th...
 2544                     destroy the free fandom honestly
 7223    Weapons stolen from National Guard Armory in N...
 4265    @wfaaweather Pete when will the heat wave pass...
 6568    Patient-reported outcomes in long-term survivo...
 Name: text, dtype: object,
 5921    0
 3904    0
 2804    1
 3718    0
 1667    0
 4435    1
 2544    1
 7223    0
 4265    1
 6568    1
 Name: target, dtype: int64)

## Text vectorization

In [56]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [57]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams = None,
                                    output_mode='int',
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [58]:
round(sum([len(sentence.split()) for sentence in train_sentences])/len(train_sentences))

15

In [59]:
max_vocab_length = 10000
max_length = 15

In [60]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length
                                 )

In [61]:
text_vectorizer.adapt(train_sentences)

In [62]:
vocab = text_vectorizer.get_vocabulary()

In [63]:
len(vocab)

10000

In [64]:
vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [65]:
vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

## Embedding 

In [66]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )

embedding

In [67]:
random_sentence = random.choice(train_sentences)

print(random_sentence)
print(embedding(text_vectorizer([random_sentence])))

No way...I can't eat that shit
tf.Tensor(
[[[ 0.03934309 -0.0433784   0.02569293 ... -0.02677385  0.0163869
   -0.01885589]
  [-0.0246877   0.03939415  0.00893077 ...  0.00760991 -0.01680835
   -0.03022722]
  [-0.0118013  -0.03864282 -0.02797339 ...  0.02499207 -0.01597313
    0.01333611]
  ...
  [-0.03885161  0.01448854  0.03522995 ...  0.02779335 -0.02621059
   -0.00666546]
  [-0.03885161  0.01448854  0.03522995 ...  0.02779335 -0.02621059
   -0.00666546]
  [-0.03885161  0.01448854  0.03522995 ...  0.02779335 -0.02621059
   -0.00666546]]], shape=(1, 15, 128), dtype=float32)


## Model 0 Naive Bayes

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [69]:
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

### Evaluate baseline model

In [70]:
baseline_score = model_0.score(val_sentences, val_labels)
baseline_score

0.7926509186351706

In [71]:
baseline_preds = model_0.predict(val_sentences)

In [72]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision, model_recall, model_fscore, _ = precision_recall_fscore_support(y_true,y_pred)
  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1':model_fscore}

  return model_results


In [73]:
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': array([0.74806202, 0.88617886]),
 'recall': array([0.93236715, 0.62643678]),
 'f1': array([0.83010753, 0.73400673])}

In [74]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = 'model_logs'

## Model 1 Simple dense model

In [87]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name='model_1_dense')

In [88]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [89]:
model_1.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [90]:
model_1_history=model_1.fit(x=train_sentences, y=train_labels, epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20230419-180807
Epoch 1/5
215/215 [==============================] - 6s 24ms/step - loss: 0.5560 - accuracy: 0.7860 - val_loss: 0.5193 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3936 - accuracy: 0.8530 - val_loss: 0.4623 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3137 - accuracy: 0.8843 - val_loss: 0.4514 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2610 - accuracy: 0.9025 - val_loss: 0.4597 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2211 - accuracy: 0.9212 - val_loss: 0.4730 - val_accuracy: 0.7835


In [84]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.6352 - accuracy: 0.6455


[0.6351858377456665, 0.6454943418502808]

In [86]:
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 6ms/step


(762, 15, 1)